In [32]:
import pickle
import os
from PIL import Image
import cv2
import dlib
import numpy as np
import face_recognition
import matplotlib.pyplot as plt

In [2]:
data_path = '../../data/ckp_with_img_geometry_106.pkl'
if not os.path.isfile(data_path):
    raise FileNotFoundError()
with open(data_path, 'rb') as f:
    data = pickle.load(f)

In [39]:
cnt = 0
for i in range(10):
    cnt += len(data[i]['labels'])
print(cnt)

309


In [3]:
print(type(data[0]['img'][0]))
# img = cv2.cvtColor(data[0]['img'][0], cv2.COLOR_RGB2GRAY)
cv2.namedWindow('img', 2)
cv2.imshow('img', data[0]['img'][0])
cv2.waitKey(0)
cv2.destroyAllWindows()

<class 'numpy.ndarray'>


In [8]:
landmarks_mat_path = '../../resource/shape_predictor_68_face_landmarks.dat'
save_path = '../../data/ck+_contains_imgs_landmarks.pickle'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(landmarks_mat_path)
def precess_image(img):
    rects = detector(img, 0)
    print(type(rects))
    landmarks = np.matrix([[p.x, p.y] for p in predictor(img, img).parts()])
    points = []
    for idx, point in enumerate(landmarks):
        # 68点的坐标
        pos = (point[0, 0], point[0, 1])
        points.append(np.array(pos))

        # 利用cv2.circle给每个特征点画一个圈，共68个
        cv2.circle(img, pos, 2, color=(0, 255, 0))
        # 利用cv2.putText输出1-68
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(idx + 1), pos, font, 0.1, (0, 0, 255), 0, cv2.LINE_AA)
    
    # 裁剪图片
    # (x, y, w, h) = rect_to_bbox(rects[i])
    # img = img[y: y + h, x: x + w]
# landmark 的边
# neighbor_edges = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10),
# (10, 11), (11,12), (12, 13), (13, 14), (14, 15), (15, 16),  # 轮廓
# (17, 18), (18, 19), (19, 20), (20, 21), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 37),  # 左眼
# ]

    points = np.stack(points[17:], axis=1)     
    return points

    # 展示
    # cv2.namedWindow("img", 2)
    # cv2.imshow("img", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [36]:
def precess_image_(img):
    landmarks = face_recognition.face_landmarks(img, model='large')
    points = []
    if len(landmarks) < 1:
        print(landmarks)
        plt.imshow(img, cmap='gray')
    print(landmarks[0])
    for key, val in landmarks[0].items():
        if key != 'chin':
            points.extend(val)
    return points

In [37]:
lm = precess_image_(data[0]['img'][0])
print(len(lm))

{'chin': [(-1, 36), (-2, 53), (-1, 70), (2, 87), (8, 102), (16, 116), (28, 128), (42, 134), (60, 137), (77, 135), (93, 130), (107, 120), (117, 106), (124, 89), (128, 72), (130, 54), (130, 37)], 'left_eyebrow': [(2, 9), (12, 2), (25, 2), (38, 5), (50, 11)], 'right_eyebrow': [(74, 11), (87, 4), (102, 0), (116, 2), (127, 11)], 'nose_bridge': [(61, 25), (61, 37), (61, 50), (61, 62)], 'nose_tip': [(46, 72), (53, 74), (61, 76), (69, 75), (76, 72)], 'left_eye': [(11, 29), (20, 22), (31, 23), (41, 30), (31, 33), (20, 33)], 'right_eye': [(82, 31), (93, 23), (104, 23), (113, 30), (105, 34), (93, 34)], 'top_lip': [(37, 104), (44, 92), (54, 87), (61, 89), (68, 87), (78, 93), (86, 105), (81, 104), (68, 96), (61, 96), (54, 96), (41, 104)], 'bottom_lip': [(86, 105), (78, 118), (68, 123), (60, 124), (52, 123), (43, 117), (37, 104), (41, 104), (53, 110), (60, 111), (68, 110), (81, 104)]}
55


In [38]:
folds = 10
for f_num in range(folds):
    data[f_num]['landmark'] = []
    for img in data[f_num]['img']:
        points = precess_image_(img)
        data[f_num]['landmark'].append(points)

{'chin': [(-1, 36), (-2, 53), (-1, 70), (2, 87), (8, 102), (16, 116), (28, 128), (42, 134), (60, 137), (77, 135), (93, 130), (107, 120), (117, 106), (124, 89), (128, 72), (130, 54), (130, 37)], 'left_eyebrow': [(2, 9), (12, 2), (25, 2), (38, 5), (50, 11)], 'right_eyebrow': [(74, 11), (87, 4), (102, 0), (116, 2), (127, 11)], 'nose_bridge': [(61, 25), (61, 37), (61, 50), (61, 62)], 'nose_tip': [(46, 72), (53, 74), (61, 76), (69, 75), (76, 72)], 'left_eye': [(11, 29), (20, 22), (31, 23), (41, 30), (31, 33), (20, 33)], 'right_eye': [(82, 31), (93, 23), (104, 23), (113, 30), (105, 34), (93, 34)], 'top_lip': [(37, 104), (44, 92), (54, 87), (61, 89), (68, 87), (78, 93), (86, 105), (81, 104), (68, 96), (61, 96), (54, 96), (41, 104)], 'bottom_lip': [(86, 105), (78, 118), (68, 123), (60, 124), (52, 123), (43, 117), (37, 104), (41, 104), (53, 110), (60, 111), (68, 110), (81, 104)]}
{'chin': [(-2, 33), (-2, 51), (-1, 70), (1, 87), (7, 103), (16, 117), (28, 127), (44, 133), (62, 134), (80, 134), (9

IndexError: list index out of range

In [10]:
for f_num in range(folds):
    print(len(data[f_num]['img']), len(data[f_num]['landmark']))

26 26
29 29
30 30
33 33
35 35
31 31
31 31
32 32
31 31
31 31


In [11]:
for lm in data[0]['landmark']:
    print(lm)

None
None
None
None
None
None
None
None
None
None
None
[[  1  10  25  40  53  79  93 107 121 128  67  68  69  70  52  60  69  77
   84  16  26  37  47  37  26  85  96 106 115 107  96  41  50  61  69  78
   91  99  92  80  70  61  50  46  61  69  79  96  79  69  61]
 [ 15   3  -2   0   6   4  -1  -2   3  15  21  33  42  52  65  67  69  67
   65  26  21  21  28  28  28  27  21  21  26  28  28  93  82  77  79  77
   81  92 107 114 115 115 107  93  85  86  85  92 104 105 104]]
[[  1   9  23  37  51  73  87 101 115 125  63  63  64  64  49  56  64  72
   79  16  25  35  45  35  24  83  93 103 111 104  94  35  46  57  63  71
   83  94  83  71  63  55  45  39  56  63  71  90  71  63  56]
 [ 15   3   1   3   9   9   3   0   1  11  26  39  52  64  75  77  78  77
   75  30  24  24  32  34  34  32  24  24  30  34  34  99  93  90  92  91
   94  99 105 109 109 108 105  98  98  99  98  99  99 100  99]]
[[  1  11  26  40  54  73  87 102 116 126  64  65  65  66  51  58  65  73
   79  16  25  35  45  35

In [15]:
print(data[0].keys())

dict_keys(['labels', 'img', 'landmark'])


In [16]:
with open(save_path, 'wb') as pfile:
    pickle.dump(
        [{
            'img': data[i]['img'],
            'labels': data[i]['labels'],
            'landmark': data[i]['landmark']
        } for i in range(folds)],
        pfile, pickle.HIGHEST_PROTOCOL
    )
    

In [17]:
with open(save_path, 'rb') as file:
    process = pickle.load(file)

In [18]:
print(len(process))

10


In [19]:
for i in range(10):
    print(len(process[i]['img']), len(process[i]['landmark']))

26 26
29 29
30 30
33 33
35 35
31 31
31 31
32 32
31 31
31 31
